#  One shot speaker verification 
- Get this repo: https://github.com/julik43/Online-Identification-of-New-Speakers
            
- Download "2weights.ckpt.data-00000-of-00001" from: http://calebrascon.info/oneshotid/2weights.ckpt.data-00000-of-00001 and move it to VGG11_EmphSpec folder
- put  *identification_system_PG.py* in the directory Online-Identification-of-New-Speakers


In [1]:
import identification_system_PG as ISPG
import pickle
import glob

Instructions for updating:
Please switch to tf.train.get_or_create_global_step
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



see previous notebooks for **data_gold** was created

In [25]:
filename_header = '~/DEMO_DATA/'
## path to pkl file that contains datframe with non overlapping data

path_gold = filename_header+'df_data/x.pkl'
data_gold = pickle.load(open(path_gold,'rb'))


In [4]:
df_all, df_nooverlap,speaker_assingments, idx__cut_ranges_original,idx_speakerlabels_artif, artif_segment_filepaths = data_gold


In [5]:
df_nooverlap.head(3)

,start,stop,speaker,spoken,spoken_clean,start_reset,stop_reset,duration
5,432.812,434.017,spkr1:,kinetic battery.,kinetic battery.,6.534,7.739,1.205
6,434.307,436.552,spkr1:,And this costs us three euros.,And this costs us three euros.,8.029,10.274,2.245
10,439.034,440.489,spkr1:,And %uh,And %uh,12.756,14.211,1.455


In [6]:
def create_train_val_set(data_gold,speaker_label,num_utterences = 3):
    df_all, df_nooverlap,speaker_assingments, idx__cut_ranges_original,idx_speakerlabels_artif, artif_segment_filepaths = data_gold

    spkr_df = df_nooverlap[df_nooverlap.speaker ==speaker_label]
    train_df_spkr = spkr_df.sort_values(by=['duration'], ascending=False).head(num_utterences)
    validation_df_spkr=spkr_df.sort_values(by=['duration'], ascending=False).iloc[num_utterences:]
    
    return train_df_spkr, validation_df_spkr


    

In [7]:
def validate_speaker_reference(validation_df_spkr, num_verified = None):
    
    all_pred_labs = []
    all_pred_probs = []

    # print(validation_df_spkr.shape)
    if num_verified ==None:
        num_verified = validation_df_spkr.shape[0]
        
    
    for item in validation_df_spkr.index[:num_verified]:
        try:
            y = glob.glob(filename_header+'single_phrases/'+str(item)+'_*.wav')
            audio_file_path = y[0]
            lab_pred, prob_pred = ISPG.classify_segment_PG(audio_file_path )
            all_pred_labs.append(lab_pred)
            all_pred_probs.append(prob_pred)
        except TypeError:
            all_pred_labs.append(None)
            all_pred_probs.append(None)

            
    
    out_df = validation_df_spkr.head(num_verified)
    out_df['label_pred'] = all_pred_labs
    out_df['probability_pred'] = all_pred_probs
    # print(out_df.shape)
    
    return out_df




In [8]:
def create_speaker_reference(train_df_spkr, label ):
    
    for item in train_df_spkr.index:
        y = glob.glob(filename_header+'single_phrases/'+str(item)+'_*.wav')
        audio_file_path = y[0]
        ISPG.classify_label_correct(audio_file_path, add_label=True, label = label )
        


In [9]:
# speaker 4 total non overlapping duration

In [10]:
df_nooverlap[df_nooverlap.speaker =='spkr4:'].duration.sum()

139.93399999999963

In [11]:
spkr4_df = df_nooverlap[df_nooverlap.speaker =='spkr4:']
spkr4_df.head(3)

,start,stop,speaker,spoken,spoken_clean,start_reset,stop_reset,duration
51,510.233,511.531,spkr4:,((the hand dynamo)),((the hand dynamo)),83.955,85.253,1.298
65,539.104,541.254,spkr4:,"We <contraction e_form=""[can't=>cannot]"">can't...",We they're all {laugh},112.826,114.976,2.150
82,569.211,569.821,spkr4:,Mhm.,Mhm.,142.933,143.543,0.610


In [12]:
spkr4_df.sort_values(by=['duration'], ascending=False).head(3)

,start,stop,speaker,spoken,spoken_clean,start_reset,stop_reset,duration
409,1040.478,1046.567,spkr4:,is something which comes after you ((b-)) -- I...,is something which comes after you ((b-)) -- I...,614.200,620.289,6.089
769,1712.245,1717.774,spkr4:,This is quite cool. So so this does not say an...,This is quite cool. So so this does not say an...,1285.967,1291.496,5.529
307,878.038,882.427,spkr4:,remote to the existing remotes for the ~T ~V. ...,remote to the existing remotes for the ~T ~V. ...,451.760,456.149,4.389


In [13]:
num_utterences = 3 # for train set

In [14]:
# train set speaker total duration
spkr4_df.sort_values(by=['duration'], ascending=False).head(num_utterences).duration.sum()

16.006999999999948

## create train validation set

In [15]:
train_df_sp4, validation_df_sp4 = create_train_val_set(data_gold,speaker_label='spkr4:',num_utterences = 3)

In [16]:
train_df_sp4.head()

,start,stop,speaker,spoken,spoken_clean,start_reset,stop_reset,duration
409,1040.478,1046.567,spkr4:,is something which comes after you ((b-)) -- I...,is something which comes after you ((b-)) -- I...,614.200,620.289,6.089
769,1712.245,1717.774,spkr4:,This is quite cool. So so this does not say an...,This is quite cool. So so this does not say an...,1285.967,1291.496,5.529
307,878.038,882.427,spkr4:,remote to the existing remotes for the ~T ~V. ...,remote to the existing remotes for the ~T ~V. ...,451.760,456.149,4.389


In [17]:
validation_df_sp4.head(3)

,start,stop,speaker,spoken,spoken_clean,start_reset,stop_reset,duration
440,1108.996,1113.241,spkr4:,But then if you have the scroll and you <contr...,But then if you have the scroll and you don't ...,682.718,686.963,4.245
460,1145.799,1149.684,spkr4:,"in the market, and probably you can go with th...","in the market, and probably you can go with th...",719.521,723.406,3.885
306,874.230,877.884,spkr4:,Well tell me one thing. We are anyway position...,Well tell me one thing. We are anyway position...,447.952,451.606,3.654


In [18]:
train_df_sp3, validation_df_sp3 = create_train_val_set(data_gold,speaker_label='spkr3:',num_utterences = 3)

## Create speaker reference 

In [19]:
create_speaker_reference(train_df_sp4, label = 'spkr4' )

INFO:tensorflow:Restoring parameters from VGG11_EmphSpec/2weights.ckpt
INFO:tensorflow:Restoring parameters from VGG11_EmphSpec/2weights.ckpt
INFO:tensorflow:Restoring parameters from VGG11_EmphSpec/2weights.ckpt


In [20]:
create_speaker_reference(train_df_sp3, label = 'spkr3' )

INFO:tensorflow:Restoring parameters from VGG11_EmphSpec/2weights.ckpt
INFO:tensorflow:Restoring parameters from VGG11_EmphSpec/2weights.ckpt
INFO:tensorflow:Restoring parameters from VGG11_EmphSpec/2weights.ckpt


## Validate results

In [21]:
sprk_4_df = validate_speaker_reference(validation_df_sp4,5)

INFO:tensorflow:Restoring parameters from VGG11_EmphSpec/2weights.ckpt
INFO:tensorflow:Restoring parameters from VGG11_EmphSpec/2weights.ckpt
INFO:tensorflow:Restoring parameters from VGG11_EmphSpec/2weights.ckpt
INFO:tensorflow:Restoring parameters from VGG11_EmphSpec/2weights.ckpt
INFO:tensorflow:Restoring parameters from VGG11_EmphSpec/2weights.ckpt


/Users/pari/University/unienv/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/pari/University/unienv/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [22]:
sprk_3_df = validate_speaker_reference(validation_df_sp3,5)

INFO:tensorflow:Restoring parameters from VGG11_EmphSpec/2weights.ckpt
INFO:tensorflow:Restoring parameters from VGG11_EmphSpec/2weights.ckpt
INFO:tensorflow:Restoring parameters from VGG11_EmphSpec/2weights.ckpt
INFO:tensorflow:Restoring parameters from VGG11_EmphSpec/2weights.ckpt
INFO:tensorflow:Restoring parameters from VGG11_EmphSpec/2weights.ckpt


/Users/pari/University/unienv/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/pari/University/unienv/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [23]:
sprk_3_df

,start,stop,speaker,spoken,spoken_clean,start_reset,stop_reset,duration,label_pred,probability_pred
521,1259.805,1263.899,spkr3:,The touch display itse- -- turns itself off to...,The touch display itse- -- turns itself off to...,833.527,837.621,4.094,spkr3,0.990543
515,1248.220,1252.239,spkr3:,is that the display contains solar cells under...,is that the display contains solar cells under...,821.942,825.961,4.019,spkr3,0.955253
524,1269.619,1272.484,spkr3:,which makes it +considerably cheaper than,which makes it +considerably cheaper than,843.341,846.206,2.865,spkr3,0.976374
509,1233.408,1236.071,spkr3:,"All right. It measures four by six centimeters,","All right. It measures four by six centimeters,",807.130,809.793,2.663,spkr3,0.888654
523,1266.566,1269.138,spkr3:,and the production costs are only three euros,and the production costs are only three euros,840.288,842.860,2.572,spkr3,0.968899


In [24]:
sprk_4_df

,start,stop,speaker,spoken,spoken_clean,start_reset,stop_reset,duration,label_pred,probability_pred
440,1108.996,1113.241,spkr4:,But then if you have the scroll and you <contr...,But then if you have the scroll and you don't ...,682.718,686.963,4.245,spkr4,0.994102
460,1145.799,1149.684,spkr4:,"in the market, and probably you can go with th...","in the market, and probably you can go with th...",719.521,723.406,3.885,spkr4,0.993261
306,874.230,877.884,spkr4:,Well tell me one thing. We are anyway position...,Well tell me one thing. We are anyway position...,447.952,451.606,3.654,spkr4,0.983571
358,958.733,962.148,spkr4:,"whether <contraction e_form=""[it=>it]['s=>is]""...","whether it's voice recognition, or the scroll ...",532.455,535.870,3.415,spkr4,0.997101
147,645.668,649.034,spkr4:,"%ah, if you are going to lose a lot of things ...","%ah, if you are going to lose a lot of things ...",219.390,222.756,3.366,spkr4,0.997425
